This is the Part-3 of Deep Reinforcement Learning Notebook series.***In this Notebook I have introduced introduces our first Value-based Algorithm known as SARSA***.


The Notebook series is about Deep RL algorithms so it excludes all other techniques that can be used to learn functions in reinforcement learning and also the Notebook Series is not exhaustive i.e. it contains the most widely used Deep RL algorithms only.



Value-based algorithms evaluate state-action pairs (s, a) by learning one of the value functions — $V^π(s) or Q^π(s, a)$— and use these evaluations to select action.Learning value functions stands in contrast to REINFORCE(which we discussed in Notebook-2 in this series) in which an agent directly learns a policy which maps states to actions
The SARSA algorithm learns $Q^π(s, a)$, however other algorithms such as the Actor-Critic algorithm learns $V^π(s)$. Hence first and foremost we will discuss why learning $Q^π(s, a)$ is a good choice in this case.The SARSA algorithm consists of two core ideas. First, a technique for learning the Q-function known as Temporal Difference (TD) learning. It is an alternative to Monte Carlo sampling for estimating state or state-action values from the experiences an agent gathers in an environment.Second, a method for generating actions using the Q-function.
This also raises the question of how agents discover good actions with the help of value-function. One of the fundamental challenges in RL is how to achieve a good balance between exploiting what an agent knows and exploring the environment in order to learn more. This is known as the exploration-exploitation tradeoff, and is covered later in this notebook. We also introduce a simple approach for solving this problem — an ∊-greedy policy.



#Why a SARSA learns the Q-function instead of the V -function
Lets us first define the Q-function and the V-function.


The Q-function measures the value of state-action pairs (s, a) under a particular policy π.The value of (s, a) is the expected cumulative discounted reward from taking action a in state s, and then continuing to act under the policy π.

$V^π(s)$ measures the value of states s under a particular policy π.
The value of a state $V^π(s)$, is the expected cumulative discounted reward from that state s onwards under a specific policy π.$Q^π(s, a)$ is closely related to $V^π(s)$. $V^π(s)$ is the expectation over the Q-values for all of the actions a available in a particular state s under the policy π.

Q and V functions are defined by below equations:

![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-03%20at%208.10.25%20PM.png?raw=true)

Equations 3.1

Relation between V and Q is show below in Equation 3.2:

![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-06%20at%203.59.46%20PM.png?raw=true)

Equation 3.2

Value functions are always defined with respect to a particular policy π, which is why they are denoted with the π superscript. The reason behind it is because $Q^π(s, a)$ depends on the sequences of rewards an agent can expect to experience after taking action a in state s. The rewards depend on the future sequences of states and actions, and these depend on a policy. Different policies may generate different future action sequences given (s, a), which may result in different rewards.

# Intuition about V-function and Q-Function
Let’s first put this in context by considering the game of chess from one player's perspective. This player can be represented as a policy π. A certain configuration of the chessboard pieces is a state s. A good chess player will have an intuition about whether π certain chess positions are good or bad. $V^π(s)$ is this intuition expressed quantitatively with a number.
We can consider the next state s′ from a legal move a, calculate $V^π(s′)$ for each of these next states, and select the action leading to the best s'.

In addition to evaluating positions, a chess player will also consider many alternative moves and the likely consequences of making them. $Q^π$(s, a) provides a quantitative value for each move. This value can be used to decide on the best move (action) to make in a particular position (state).
# Prons and Cons of Q and V function
The action selection form V-function is time-consuming and relies on knowing the transition function, which is not available for many environments.

$Q^π(s, a)$ has the benefit of giving agents a direct method for acting. Agents can calculate $Q^π(s, a)$ for each of the actions a ∈ $A_s$ available in a particular state s, and select the action with the maximum value. In the optimal case, $Q^π(s, a)$ represents the optimal expected value from taking action a in state s, denoted Q(s, a). It represents the best you could possibly do if you acted optimally in all subsequent states. Knowing Q(s, a) therefore yields an optimal policy.

The disadvantage of learning $Q^π(s, a)$ is that the function approximation is more computationally complex, and requires more data to learn from when compared with $V^π(s)$. Learning a good estimate for $V^π(s)$ requires that the data covers the state space reasonably well, whereas learning a good estimate for $Q^π(s, a)$ requires that data covers all (s, a) pairs, not just all states s. The combined state-action space is often significantly larger than the state space and so more data is needed to learn a good Q-function estimate.

$V^π(s)$ maybe a simpler function to approximate but it has one significant disadvantage. To use $V^π(s)$ to choose actions, agents need to be able to take each of the available actions a ∈ $A_s$ in a state s, and observe the next state the environment transitions to $s^{′2}$, along with the next reward r. Then an agent can act optimally by selecting the action which led to the largest expected cumulative discounted rewards E[r + $V^π$ (s′)]. However, if state transitions are stochastic (taking action a in state s can result in different next states s′) the agent may need to repeat this process many times to get a good estimation of the expected value of taking a particular action. This is a potentially computationally expensive process.



# TEMPORAL DIFFERENCE LEARNING
The main idea is to use a neural network that produces Q-value estimates given (s, a) pairs as inputs. This is known as a value network.
The workflow for learning the value network parameters goes as follows: generate trajectories τ s and predict a $Q_@$ value for each (s, a) pair. Then use the trajectories to generate target Q-values $Q_{tar}$. Finally, minimize the distance between $Q_@$ and $Q_{tar}$ using a standard regression loss such as MSE (mean squared error). Repeat this process many times.

TD learning is how we will produce the target values $Q_{tar}$ in SARSA. To motivate this, it is helpful to consider how it can be done with Monte Carlo sampling(which was discussed in notebook-part-2 of this series).

Given N trajectories $τ_i$, i∈ {1, . . . , N} starting in state s, where the agent took action a, the Monte Carlo (MC) estimate of $Q^π_{tar}$
(s, a) is the average of all the trajectories’ returns shown in below Equation 3.3

![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-06%20at%204.47.59%20PM.png?raw=true)

Equation 3.3


If we have access to an entire trajectory $τ_i$, it is possible to calculate the actual Q-value the agent received in that particular case for each (s, a) pair in $τ_i$. This follows from the definition of the Q-function since the expectation of future cumulative discounted rewards from a single example is just the cumulative discounted reward from the current time step to the end of the episode for that example. Equation 3.3 shows that this is also the Monte Carlo estimate for $Q^π(s, a)$ where the number of trajectories used in the estimate is equal to N = 1. Now each (s, a) pair in the dataset is associated with a target Q- value. The dataset can be said to be “labeled” since each datapoint (s, a) has a corresponding target value $Q^π_{tar}(s, a)$.

One disadvantage of Monte Carlo sampling is that an agent has to wait for episodes to end before any of the data from that episode can be used to learn from. This is evident from Equation 3.3. To calculate $Q^π_{tarMC}(s, a)$, the rewards for the remainder of the trajectory starting from (s, a) are required. Episodes might have many time steps as measured by T, delaying training. This motivates an alternative approach to learning Q-values, TD learning.

The key insight in TD learning is that Q-values for the current time step can be defined in terms of Q-values of the next time step. That is, $Q^π(s, a)$ is defined recursively as shown below in Equation 3.4
![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-06%20at%204.53.45%20PM.png?raw=true)

Equation 3.4

The equation 3.4 is known as Bellman equation.If the Q- function is correct for the policy π Equation 3.4 holds exactly. It also suggests a way of learning Q-values. We just saw how Monte Carlo sampling can be used to learn $Q^π(s, a)$ given many trajectories of experiences. The same idea can be applied here, using TD learning to derive the target value $Q^π_{tar}(s,a)$  for each (s, a) pair.Assume we have a neural network to represent the Q-function.In TD learning, $Q^π_{tar}(s_t,a_t)$ is derived by estimating the right hand side of Equation 3.4 using the Q-function(obtained by neural network).Each training iteration, $Q^π_@(s_t,a_t)$ is updated to bring it closer to $Q^π_{tar}(s_t,a_t)$.

# Problems with Equation 3.4
There are two problems with Equation 3.4 if it is to be used to construct $Q^π_{tar}(s_t,a_t)$— the two expectations. 

The first problem is the outer expectation. This can be illustrated by supposing we had collections of trajectories, {$τ_1, τ_2, . . . , τ_M$}. Each trajectory $τ_i$ contains (s, a, r, s′) tuples. For each tuple (s, a, r, s′), only one example of the next state s′, is available given the action selected.
If the environment is deterministic then it is correct to only consider the actual next state when calculating the expectation over the next states. However, if the environment is stochastic, this breaks down. Taking action a in states s could transition the environment into several different next states, but only one next state was actually observed at each step. This problem can be overcome by considering only one example — the one that actually happened. This does mean that the Q-value estimate may have high variance if the environment is stochastic, but helps make the estimation more tractable. Using just one example to estimate the distribution over the next states s′ and rewards s, we can take the outer expectation out and the Bellman equation can be rewritten as shown below in Equation 3.5.


![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-06%20at%205.09.47%20PM.png?raw=true)

Equation 3.5

The second problem is the inner expectation of Equation 3.4 over the actions. We have access to the Q-value estimates for all of the actions in the next state s′ because we are using the current estimate of the Q-function to calculate the values. The problem arises from not knowing the probability distributions over actions that are needed to calculate the expectation. There are two ways to solve this, each of which corresponds to a different algorithm; SARSA and DQN(DQN will be discussed in Notebook-4 of this series).SARSA’s solution is to use the action actually taken in the next state, a′ (Equation 3.6). Over the course of many examples, the proportion of actions selected should approximate the
probability distribution over actions, given states. DQN’s
solution is to use the maximum valued Q-value (Equation 3.7).
A third alternative is to derive the probability distribution over actions from the policy and calculate the expectation. This algorithm is known as expected SARSA. This corresponds to an implicit policy of selecting the action with the maximum Q-value with probability 1. As we will see in
Notebook-4 the implicit policy in Q-learning is greedy with
respect to Q-values, and so the transformation of the equation is valid.
![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-06%20at%205.14.04%20PM.png?raw=true)

Equation 3.6

![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-06%20at%205.14.14%20PM.png?raw=true)

Equation 3.7

Now  $Q^π_{tar}(s_t,a_t)$ can be calculated using the right hand side Equation 3.6 for each tuple (s,a,r,s,a)

Notice that only the information from the next step in the environment was required to calculate the target Q-values instead of a whole episode’s trajectory. This makes it possible to update the Q-function in a batched (after a few examples) or online (after one example) manner when using TD learning instead of having to wait for an entire trajectory to finish.
TD learning is a bootstrapped learning method because the existing estimate of Q-values for  (s, a) pair are used when calculating $Q^π_{tar}(s_t,a_t)$. This has the advantage of lowering the variance of the estimate when compared to Monte Carlo sampling. TD learning only uses the actual reward r, from the next state s′, and combines it with Q-value estimates to approximate the remaining part of the value. The Q-function represents an expectation over different trajectories and so typically has a lower variance than the whole trajectory used by Monte Carlo sampling. However, it also introduces bias into the learning approach since the Q-function approximator is not perfect.

# ACTION SELECTION IN SARSA 
TD learning gives us a method for learning how to evaluate actions. What is missing is a policy, a mechanism for selecting actions. Suppose we had already learned the optimal Q-function. Then the value of each state-action pair will represent the best possible expected value from taking that action. This gives us a way to reverse engineer the optimal way of acting. If the agent always selects the action which corresponds to the maximum Q-
value in each state, if the agent acts greedily with respect to the Q-values, then it will be acting optimally. This also implies that the SARSA algorithm is limited to discrete action spaces(To identify the maximum Q-value in state s, we need to compute the Q-values for all the possible actions in that state. When actions are discrete, this is straightforward because we can list all the possible actions and compute their Q-value. However, when actions are continuous, the actions cannot be fully enumerated, and this becomes a problem. For this reason, SARSA and other value-based methods are generally restricted to only discrete action spaces. However, there are methods such as QT– Opt which approximate the maximum Q-value by sampling from a continuous action space. We will not cover QT-Opt in this series).


Unfortunately, the optimal Q-function is typically not known. However, the relationship between the optimal Q-function and acting optimally tells us that if we have a good Q-function then we can derive a good policy. It also suggests an iterative approach to improving the Q-value.
First, randomly initialize a neural network with parameters θ to represent the Q-function, denoted $Q^π(s, a; θ)$ Then repeat the following steps until the agent stops improving, as evaluated by the total un-discounted5 cumulative rewards received during an episode.
1. Use $Q^π(s, a; θ)$ to act in the environment, by acting greedily with respect to the Q-values. Store all of the (s, a, r, s′) experiences.
2. Use the stored experiences to update Qπ(s, a; θ) using the SARSA Bellman equation (Equation 3.6). This improves the Q-function estimate, which in turn improves the policy since the Q-value estimates are now better.

# Exploration and Exploitation
One issue with acting greedily with respect to the Q-values is that it is deterministic, which means that an agent may not explore the entire state-action space sufficiently. If an agent always selects the same action a in state s, then there may be many (s, a) pairs that an agent never experiences. This can result in the Q-function estimates for some (s, a) pairs being inaccurate since they were randomly initialized and there were no experiences of (s, a). So there is no opportunity for a network to learn about this part of the state-action space. Consequently, the agent may make suboptimal actions and get stuck in local minima.

To mitigate this issue, it is common to use an ∊-greedy policy instead of a purely greedy policy for the agent. Under this policy, an agent selects the greedy action with probability 1 – ∊, and acts randomly with probability ∊. So ∊ is known as the exploration probability since the acting randomly ∊× 100% of the time helps an agent explore the state-action space. Unfortunately, this comes at a cost. Now the policy may not be as good as the greedy policy because the agent is taking random actions with non-zero probability instead of the Q-value maximizing action.


The tension between the potential value of exploring during training and taking the best actions based on the information available to an agent (Q-values) is known as the exploration-exploitation trade-off. A common approach to managing this trade-off is to start with high ∊, 1.0, or close to this so that the agent acts almost randomly and rapidly explores the state-action space. Also, the agent has not learned anything at the beginning of training so there is nothing to exploit. Overtime ∊ is gradually decayed so that after many steps the policy hopefully approaches the optimal policy. As the agent learns better Q-function and so better policies, there is less benefit to exploring and the agent should act more greedily. In the ideal case, the agent will have discovered the optimal Q- function after some time and so ∊ can be reduced to 0. In practice, due to limited time, continuous or high dimensional discrete state spaces, and non-linear function approximators the learned Q-function does not converge to the optimal policy. Instead ∊ is typically annealed to a small value, 
0.1 – 0.001, for example, and fixed to allow a small amount of continued exploration.



# **SARSA ALGORITHM**
![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-07%20at%207.07.44%20PM.png?raw=true)



# **IMPLEMENTING SARSA**

Below is the implementation in code of this algorithm on a simple example of Cliff-Walking where agent tries to 

Below code setups the environment required to run the game.

In [ ]:
#Installing the dependencies
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y

In [ ]:
!pip install gym[atari]
!pip install pyvirtualdisplay
!pip install piglet

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
from IPython import display
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython.display import clear_output

from IPython import display as ipythondisplay

In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

This part ensures the reproducibility of the code below by using a random seed and setups the environment.

In [ ]:
import gym.envs.toy_text 
RANDOM_SEED=1
N_EPISODES=500

# random seed (reproduciblity)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# set the env
env=gym.envs.toy_text.CliffWalkingEnv() # env to import
#env=wrap_env(env)
env.seed(RANDOM_SEED)
env.reset() # reset to env

Defining the SARSA Class
At initiation, the SARSA object sets a few parameters. First, is the environment in which the model learns and its properties. Second, are both the parameters of the REINFORCE algorithm — Gamma (𝛾) , alpha (𝛼) and epsilon(∊). Gamma, is the decay rate of past observations,alpha is the learning rate by which the gradients affect the policy update and epsilon is the value used for ∊-greedy policy.
Lastly, it sets the learning rate for the neural network.

In [ ]:
class Sarsa:

  def __init__(self, env, path=None):
    self.env=env #import env
    self.state_shape=(1,) # the state space
    self.action_shape=env.action_space.n # the action space
    self.gamma=[0.99] # decay rate of past observations
    self.alpha=1e-4 # learning rate in the policy gradient
    self.learning_rate=0.01 # learning rate in deep learning
    self.epsilon_initial_value=1.0 # initial value of epsilon
    self.epsilon_current_value=1.0 # current value of epsilon
    self.epsilon_final_value=0.0001 # final value of epsilon
  
    if not path:
      self.model=self._create_model() #build model
    else:
      self.model=load_model(path) #import model

Creating a Neural Network Model
I chose to use a neural network to implement this agent. The network is a simple feedforward network with a few hidden layers. The output layer incorporates a softmax activation.The model takes the state as input and outputs Q(s,a) values for that state following the current policy.

In [ ]:

  def _create_model(self):
    ''' builds the model using keras'''
    model=Sequential()

    # input shape is of observations
    model.add(Dense(64, activation="relu",input_shape=self.state_shape))
    # add a relu layer 
    model.add(Dense(32, activation="relu"))

    # output shape is according to the number of action
    # The softmax function outputs a probability distribution over the actions
    model.add(Dense(env.action_space.n, activation="softmax")) 
    model.compile(loss="CategoricalCrossentropy",
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))
    return model

Action Selection
The get_action method guides its action choice. It uses ∊-greedy policy for choosing the action. Initially, since the value of ∊ is high agent choose randomly but later in training the agent chooses the actions that produces maximum Q value.

In [ ]:
def get_action(self, state):
        '''samples the next action based on the E-greedy policy'''

        state=int(state)
        state=np.array([state],dtype='float32')
        state=tf.convert_to_tensor(state)
        x=random.random()
        if x < self.epsilon_current_value: 
          action=random.randrange(0,4) #Exlporation
          q_values=self.model.predict(state)
          
          q_act=q_values[0][action]

        else:
          q_values=self.model.predict(state) #Exploitation
          max_Q = np.argmax(q_values)
          action = max_Q
          q_act=q_values[0][action]

        return action

Updating the Policy
Following each timestep, the model uses the data collected to update the policy parameters. Here, training the neural network updates the policy.To update the network we first calculate the loss.First $Q_@(s,a)$ is calculated for the action a taken by the agent in the current state s for each experience in the batch. This involves a forward pass through the value network to obtain the Q-value estimates for all of the (s, a) pairs. Recall that in the SARSA version of the Bellman equation, we use the action actually taken by the agent in the next state to calculate.So we repeat the same step with next states instead of states.Next, we extract the Q-value estimate for the action actually taken in both cases.With this, we can estimate $Q_{tar}(s,a)$.

One important element to be aware of is for each experience, the agent only received information about the action actually taken. Consequently it can only learn something about this action, not the other actions available. This is why the loss is only calculated using values which correspond to the actions taken in the current and next states.


In [ ]:
def update_policy(self,delta,state,next_state,action,next_state_action,reward):
    '''
    Updates the policy network using the NN model.
      '''

    state=int(state)
    state=np.array([state],dtype='float32')
    state=tf.convert_to_tensor(state)

    next_state=int(next_state)
    next_state=np.array([next_state],dtype='float32')
    next_state=tf.convert_to_tensor(next_state)
    preds=(self.model.predict(next_state))          
    next_state_q_act=preds[0][next_state_action]      #This is 𝑄𝑡𝑎𝑟(𝑠,𝑎) 
    target=reward+np.asarray(delta)*np.asarray(self.gamma)*next_state_q_act
    target=np.reshape(target,(1,1))

    mse=tf.keras.losses.MeanSquaredError()
    optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)

    def train_step(state, target):
      with tf.GradientTape() as tape:
        preds= (self.model)(state,training=True) 
        logits=(preds[0][action])                   #This is the 𝑄@(𝑠,𝑎)
        logits=tf.reshape(logits,(1,1))
        loss=mse(target,logits)
        print(loss)
      grads = tape.gradient(loss,self.model.trainable_variables)
      optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
    train_step(state,target)

Training the model
This method creates a training environment for the model. Iterating through a set number of episodes, it uses the model to sample actions and play them. When such a timestep ends, the model is using the observations to update the policy.

In [ ]:
def train(self, episodes):
    '''
          train the model
          episodes - number of training iterations 
          ''' 
    env=self.env
    total_rewards=np.zeros(episodes)

    for episode in range(episodes):
      # each episode is a new game env
      state=env.reset()
      done=False          
      episode_reward=0 #record episode reward
      action=self.get_action(state)
      print("Episode Started")
      while not done:
        # play an action and record the game state & reward per episode
        next_state, reward, done, _=env.step(action)
        if done:
          delta=0.0
        else:
          delta=1.0
        next_state_action=self.get_action(next_state)
        state=next_state
        print("Episode Going On.Action taken:",action)
        action=next_state_action
        episode_reward+=reward
        self.update_policy(delta,state,next_state,action,next_state_action,reward)
        total_rewards[episode]=episode_reward
      print("Episode_reward{}".format(episode_reward))
      self.model.save('model_{}.h5'.format(episode))
      print("Episode Ended")
    if self.epsilon_current_value > self.epsilon_final_value:
      self.epsilon_current_value=self.epsilon_current_value-(self.epsilon_initial_value_-self.epsilon_final_value)/1000
  
Agent=Sarsa(env)
Agent.train(episodes=200)

With the help of below code we run our algorithm and see the success of it.(Before running this set self.epsilon_current_value=0.001 in SARSA class so that model does not choose actions randomly)

In [ ]:
from tensorflow.keras.models import load_model
Agent_3=Sarsa(env,path='model.h5')

state=env.reset()
action=Agent_3.get_action(state)
done=False
while not done:
    action=Agent_3.get_action(state)
    next_state, reward, done, _=env.step(action)
    state=next_state
env.render(mode="ipython")